<a href="https://colab.research.google.com/github/ElaYJ/Study_Deep_Learning/blob/main/DL_Framework/TensorFlow/04_%EC%9E%90%EB%8F%99%EB%AF%B8%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 자동 미분

In [1]:
import tensorflow as tf

## `tf.GradientTape`

- 텐서플로의 자동 미분

- tf.GradientTape는 컨텍스트(context) 안에서 실행된 모든 연산을 테이프(tape)에 "기록".

- 그 다음 텐서플로는 후진 방식 자동 미분(reverse mode differentiation)을 사용해 테이프에 "기록된" 연산의 그래디언트를 계산합니다.


#### Scalar 를 Scalar로 미분

In [8]:
x = tf.Variable(3.0)

# tape.gradient()를 한 번만 호출 가능
with tf.GradientTape() as tape:
    y = x**2

In [9]:
y

<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

In [10]:
# dy = 2x * dx
dy_dx = tape.gradient(y, x)
dy_dx

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [11]:
dy_dx.numpy()

6.0

#### Scalar를 Vector로 미분

In [26]:
w = tf.Variable(tf.random.normal((3, 2)), name='w') #--> weights
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b') #--> bias
x = [[1., 2., 3.]]

# persistent=True 옵션으로 tape.gradient()를 한 번 이상 호출 가능
with tf.GradientTape(persistent=True) as tape:
    y = x @ w + b
    loss = tf.reduce_mean(y**2)


In [13]:
w

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[ 0.966009  ,  0.04924868],
       [-0.29810202,  1.384905  ],
       [-1.0577765 , -0.19742115]], dtype=float32)>

In [14]:
b

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>

In [16]:
y

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-2.8035245,  2.2267952]], dtype=float32)>

In [15]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=6.4091835>

In [27]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b]) #--> List Type

[dl_dw, dl_db]

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 1.6136951, -2.0941195],
        [ 3.2273903, -4.188239 ],
        [ 4.8410854, -6.2823586]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 1.6136951, -2.0941195], dtype=float32)>]

In [28]:
# 기록되고 있는 variable 확인

tape.watched_variables()

(<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 1.2125661 ,  0.33372277],
        [-0.5328175 , -0.05227595],
        [ 0.4889213 , -0.7744301 ]], dtype=float32)>,
 <tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>)

In [18]:
my_vars = {
    'w': w,
    'b': b
}

grad = tape.gradient(loss, my_vars) #--> Dict Type
grad['b']

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-2.8035245,  2.2267952], dtype=float32)>

In [19]:
grad

{'w': <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-2.8035245,  2.2267952],
        [-5.607049 ,  4.4535904],
        [-8.410574 ,  6.6803856]], dtype=float32)>,
 'b': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-2.8035245,  2.2267952], dtype=float32)>}

## 자동미분 컨트롤 하기!

  - `tf.GradientTape`은 `tf.Variable`만 기록한다.

  - A(variable) + tensor(constant) 연산은 tensor(constant)를 반환하므로 기록되지 않는다.

  - 변수(variable)인 weights를 모델에서 사용할 때,
  
    `trainable` 조건으로 미분 기록을 제어할 수 있다.

In [20]:
# A trainable variable
x0 = tf.Variable(3.0, name='x0')
x0

<tf.Variable 'x0:0' shape=() dtype=float32, numpy=3.0>

In [21]:
# Not trainable variable
x1 = tf.Variable(3.0, name='x1', trainable=False)
x1

<tf.Variable 'x1:0' shape=() dtype=float32, numpy=3.0>

In [22]:
# Not a Variable: A variable + tensor returns a tensor.
x2 = tf.Variable(2.0, name='x2') + 1.0
x2

<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [23]:
# Not a variable
x3 = tf.constant(3.0, name='x3')
x3

<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [24]:
with tf.GradientTape() as tape:
    y = (x0**2) + (x1**2) + (x2**2) + (x3**2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
    print(g)

tf.Tensor(6.0, shape=(), dtype=float32)
None
None
None


#### 기록되고 있는 variable 확인하기

In [25]:
tape.watched_variables()

(<tf.Variable 'x0:0' shape=() dtype=float32, numpy=3.0>,)